<a href="https://colab.research.google.com/github/alexeiveselov92/other_projects/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%D1%80%D1%8B%D0%BD%D0%BA%D0%B0%20%D1%82%D1%80%D1%83%D0%B4%D0%B0%20%D0%B2%20%D0%A1%D0%9F%D0%91%20%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание работы
Необходимо было провести анализ рынка труда в Санкт-Петербурге по вакансиям на hh.ru по ключевым запросам:
1. Data Scientist
2. Аналитик данных
3. Аналитик

Целью работы является выявление средней заработной платы, а также наиболее важных ключевых навыков по вакансиям.

Для проведения анализа была сделана выгрузка всех вакансий с площадки hh.ru по ключевым словам через Google Sheets с помощью XPath запросов. Далее в Google Sheets были удалены нерелевантные вакансии и значения зарплат были переведены в русские рубли. 

# Спецификация
* **`keywords`** - ключевые слова при поиске вакансий на hh.ru
* **`company`** - название компании работодателя
* **`position`** - название вакансии
* **`link`** - ссылка на вакансию
* **`salary`** - заработная плата
* **`key_skills`** - ключевые навыки, необходимые под вакансию


# Загрузка библиотек

In [2]:
import pandas as pd
import math as mth
from scipy import stats as st
import random
import numpy as np

#визуализация
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import plotly.express as px
import seaborn as sns; sns.set()

#загрузка файлов локально и с диска
from google.colab import files
from google.colab import drive
# drive.mount('/content/drive')
#чтобы помещалось все горизонтально
pd.options.display.expand_frame_repr = False

import warnings
warnings.filterwarnings('ignore')

#загрузка файла локально
#uploaded = files.upload()

#загрузка с гугл диска (надо подключиться к google диску либо через drive.mount('/content/drive') либо нажав кнопку 'Mount Drive')
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Загрузка датасета
Датасет заранее был отчищен от нерелевантных вакансий и дубликатов при выгрузке.

При выгрузке вакансий по ключевым словам по разным запросам выгружались одни и те же вакансии.

Поэтому при выгрузке одних и тех же вакансий по одним и тем же запросам присваивались ключевые слова менее частотного запроса. 

Т.е. приоритет давался сначала "Data Scientist", затем "Аналитик данных" и только потом уже "Аналитик".




In [3]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/datasets/vacancies_spb_it.csv')
df.head(5)

,keywords,company,position,link,salary,key_skills
0,аналитик данных,Fmedia,Аналитик (отдел закупок),https://spb.hh.ru/analytics_source/vacancy/367...,NaN,"MS Outlook, Internet Marketing, Закупка товаро..."
1,аналитик данных,Авито,Аналитик отдела продаж,https://spb.hh.ru/vacancy/35000627,NaN,"SQL, MS SQL, Бизнес-анализ, Экономический анал..."
2,аналитик данных,ООО ЭДГОУ,Аналитик,https://spb.hh.ru/vacancy/36812321,80000.0,"Python, SQL, Data Mining, Английский язык, Ана..."
3,аналитик данных,Бронка Групп,Товарный аналитик,https://spb.hh.ru/vacancy/36069746,60000.0,"Прогнозирование, Анализ финансовых показателей..."
4,аналитик данных,First_Line,Финансовый аналитик,https://spb.hh.ru/vacancy/36157991,40000.0,"Финансовый анализ, Управленческий учет, Анализ..."


# Посмотрим средние зарплаты по вакансиям и общую статистику

In [4]:
print('Из {} вакансий величина ЗП указана в {} случаях ({:.2%})'.format(len(df), df['salary'].count(), df['salary'].count()/len(df)))
print('Из {} вакансий ключевые навыки указаны в {} случаях ({:.2%})'.format(len(df), df['key_skills'].count(), df['key_skills'].count()/len(df)))
print('Всего компаний в выгрузке: {}'.format(len(df['company'].unique())))
grouped_by_keywords = df.groupby('keywords').agg({'company':'nunique', 'position':'count', 'salary':['count', 'mean', 'median'], 'key_skills':'count'})
grouped_by_keywords.columns = ['companies', 'positions', 'vacancies_with_salary', 'mean_salary', 'median_salary', 'vacancies_with_key_skills']
grouped_by_keywords

Из 679 вакансий величина ЗП указана в 122 случаях (17.97%)
Из 679 вакансий ключевые навыки указаны в 244 случаях (35.94%)
Всего компаний в выгрузке: 364


,companies,positions,vacancies_with_salary,mean_salary,median_salary,vacancies_with_key_skills
keywords,,,,,,
Data Scientist,29,37,12,117916.666667,100000.0,16
аналитик,210,327,53,84792.452830,70000.0,112
аналитик данных,208,315,57,78964.912281,70000.0,116


> * Всего вакансий: 679
* Всего компаний: 364
* Заработную плату указали только в 18% случаев.
* Ключевые навыки указали только в 35% случаев.
* Средняя ЗП у Data Scientist: 100000 руб.
* Средняя ЗП у аналитика данных: 70000 руб.
* Средняя ЗП у аналитика: 70000 руб.

# ТОП-15 ключевых требуемых навыков в зависимости от вакансии

In [5]:
df[['key_skills']] = df[['key_skills']].fillna('*')
df['skills_list'] = df['key_skills'].apply(lambda x: x.split(', '))
df

skills_list = []
skills_analyst = []
skills_data_analyst = []
skills_data_scientist = []

lists = {
    'аналитик':skills_analyst,
    'аналитик данных':skills_data_analyst,
    'Data Scientist':skills_data_scientist
}


for keyword in lists:
  for skills in df.query('keywords==@keyword')['skills_list']:
    for skill in skills:
      if skill!='*':
        lists[keyword].append(skill)

skills_list = skills_analyst + skills_data_analyst + skills_data_scientist
  

skills_analyst_df = pd.DataFrame({'skill':skills_analyst})
skills_data_analyst_df = pd.DataFrame({'skill':skills_data_analyst})
skills_data_scientist_df = pd.DataFrame({'skill':skills_data_scientist})
skills = pd.DataFrame({'skill':skills_list})

skills_analyst_df = pd.DataFrame(skills_analyst_df['skill'].value_counts()).reset_index()
skills_data_analyst_df = pd.DataFrame(skills_data_analyst_df['skill'].value_counts()).reset_index()
skills_data_scientist_df = pd.DataFrame(skills_data_scientist_df['skill'].value_counts()).reset_index()
skills = pd.DataFrame(skills['skill'].value_counts()).reset_index()

skills_analyst_df.columns = ['skill', 'vacancies']
skills_data_analyst_df.columns = ['skill', 'vacancies']
skills_data_scientist_df.columns = ['skill', 'vacancies']
skills.columns = ['skill', 'vacancies']

skills_analyst_df['percent_of_vacancies'] = skills_analyst_df['vacancies'] / len(df.query('key_skills !="*" and keywords=="аналитик"'))
skills_data_analyst_df['percent_of_vacancies'] = skills_data_analyst_df['vacancies'] / len(df.query('key_skills !="*" and keywords=="аналитик данных"'))
skills_data_scientist_df['percent_of_vacancies'] = skills_data_scientist_df['vacancies'] / len(df.query('key_skills !="*" and keywords=="Data Scientist"'))
skills['percent_of_vacancies'] = skills['vacancies'] / len(df.query('key_skills !="*"'))

lists_for_plots = {
    'аналитик':skills_analyst_df,
    'аналитик данных':skills_data_analyst_df,
    'Data Scientist':skills_data_scientist_df, 
    'аналитик / аналитик данных / Data Scientist':skills
}

for keyword in lists_for_plots:
  fig = px.bar(lists_for_plots[keyword].head(15), 
              x='skill', y='vacancies', width = 1300, height = 650, color='skill', 
              text='vacancies')
  fig.update_xaxes(tickangle=30)
  fig.update_layout(
      title='ТОП-15 ключевых навыков в профессии {} в СПБ'.format(keyword),
      xaxis_title="Ключевой навык",
      yaxis_title="Вакансий с навыком")
  fig.update_traces(textposition='outside')
  for trace, percent in zip(fig.data, lists_for_plots[keyword].head(15)['percent_of_vacancies']):
      trace.name = trace.name.split('=')[1] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'
  fig.show()


# ТОП-15 самых высокооплачиваемых навыков
Выведем ТОП-15 самых высокооплачиваемых навыков без разделения по вакансиям. При этом будем учитывать только те навыки, по которым есть хотябы 3 вакансии с указанной ЗП для минимальной репрезентативности.

In [6]:
def find_rows_with_word(word):
  filter_list = []
  for value in df['skills_list']:
    if word in value:
      filter_list.append(True)
    else:
      filter_list.append(False)
  return df[filter_list]

def create_new_columns(row):
  new_data = find_rows_with_word(row['skill'])
  row['vacancies_with_salary'] = new_data.agg({'salary':'count'})[0]
  row['mean_salary'] = new_data.agg({'salary':'mean'})[0]
  row['median_salary'] = new_data.agg({'salary':'median'})[0]
  return row

skills = skills.apply(create_new_columns, axis = 1)
skills_with_max_salary = skills.query('vacancies_with_salary>2 ').sort_values(by='median_salary', ascending=False)
skills_with_max_salary['median_salary'] = (skills_with_max_salary['median_salary']/1000).astype('int')
skills_with_max_salary['text_salary'] = skills_with_max_salary['median_salary'].astype('str') + ' тыс.руб.'

fig = px.bar(skills_with_max_salary.head(15), 
            x='skill', y='median_salary', width = 1300, height = 650, color='skill', 
            text='median_salary')
fig.update_xaxes(tickangle=30)
fig.update_layout(
    title='ТОП-15 самых оплачиваемых навыков в профессиях аналитик / аналитик данных / Data Scientist в СПБ'.format(keyword),
    xaxis_title="Ключевой навык",
    yaxis_title="Средняя заработная плата, тысяч рублей")
fig.update_traces(textposition='inside')
fig.update_yaxes(ticksuffix=" тыс.руб. ")
for trace, percent in zip(fig.data, skills_with_max_salary.head(15)['percent_of_vacancies']):
    trace.name = trace.name.split('=')[1] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'
fig.show()

# ТОП-15 наиболее частых названий должности аналитика

In [9]:
df.sort_values(by='salary', ascending=False).head(15)


lists_for_plots = {
    'аналитик':skills_analyst_df,
    'аналитик данных':skills_data_analyst_df,
    'Data Scientist':skills_data_scientist_df, 
    'аналитик / аналитик данных / Data Scientist':skills
}
for column in ['аналитик', 'аналитик данных', 'Data Scientist']:
  most_frequent_job_titles =  pd.DataFrame({'frequency':df.query('keywords==@column')['position'].value_counts()}).reset_index()
  most_frequent_job_titles.columns = ['vacancy', 'frequency']
  most_frequent_job_titles['percent_of_vacancies'] = most_frequent_job_titles['frequency'] / len(df.query('keywords==@column'))
  fig = px.bar(most_frequent_job_titles.head(15), 
              x='vacancy', y='frequency', width = 1300, height = 650, color='vacancy', 
              text='frequency')
  fig.update_xaxes(tickangle=30)
  fig.update_layout(
      title='ТОП-15 наиболее частых названий должности "{}" в СПБ'.format(column),
      xaxis_title="Название вакансии",
      yaxis_title="Частота названия вакансии")
  fig.update_traces(textposition='outside')
  for trace, percent in zip(fig.data, most_frequent_job_titles.head(15)['percent_of_vacancies']):
      trace.name = trace.name.split('=')[1] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'
  fig.show()

most_frequent_job_titles =  pd.DataFrame({'frequency':df['position'].value_counts()}).reset_index()
most_frequent_job_titles.columns = ['vacancy', 'frequency']
most_frequent_job_titles['percent_of_vacancies'] = most_frequent_job_titles['frequency'] / len(df)

fig = px.bar(most_frequent_job_titles.head(15), 
            x='vacancy', y='frequency', width = 1300, height = 650, color='vacancy', 
            text='frequency')
fig.update_xaxes(tickangle=30)
fig.update_layout(
    title='ТОП-15 наиболее частых названий должности "аналитик / аналитик данных / Data Scientist" в СПБ'.format(keyword),
    xaxis_title="Название вакансии",
    yaxis_title="Частота названия вакансии")
fig.update_traces(textposition='outside')
for trace, percent in zip(fig.data, most_frequent_job_titles.head(15)['percent_of_vacancies']):
    trace.name = trace.name.split('=')[1] + ' (' + '{:.2%}'.format(percent) + ' вакансий)'
fig.show()

# Вывод
На момент проведения анализа рынка (01.05.2020) всего в Санкт-Петербурге по вакансиям аналитика, аналитика данных и Data Scientist было **679** открытых релевантных позиций в **364** компаниях. 

Из всего этого числа позиций заработную плату указали только **в 18% случаев**, а ключевые навыки **в 35% случаев**.

> Средние заработные платы находятся на следующем уровне:
* **`Data Scientist`**: **100000 руб.**
* **`Аналитик данных`**: **70000 руб.**
* **`Аналитик`**: **70000 руб.**

> В зависимости от типа вакансии приоритеты ключевых навыков меняются, но можно выделить основные навыки, которые всегда одинаково важны:
1. **`SQL`**
2. **`Английский язык (принципиальное понимание)`**
3. **`Python`**
4. **`Бизнес-анализ`**
5. **`Статистический анализ`**

> Самыми высокооплачиваемыми навыками, исходя из указанной информации работодателем, в профессии оказались:
1. **`Big Data`** - средняя ЗП 120 тыс.руб.
2. **`Английский — B2 — Средне-продвинутый`** - средняя ЗП 115 тыс.руб.
3. **`Linux`** - средняя ЗП 110 тыс.руб.
4. **`SQL`** - средняя ЗП 100 тыс.руб.
5. **`Аналитика`** - средняя ЗП 100 тыс.руб.
10. При этом **`Python`** стоит на 10-м месте со средней ЗП 90 тыс.руб.

> Наиболее частые названия вакансий должности аналитика:
1. **`Системный аналитик`** - 45 раз
2. **`Аналитик`** - 23 раз
3. **`Бизнес-аналитик`** - 21 - раз
4. **`Маркетолог-аналитик`** - 13 раз
5. **`Финансовый аналитик`** - 8 раз
6. **`Продуктовый аналитик`** - 6 раз
7. **`Аналитик BI`** - 6 раз
8. **`Data Scientist`** - 5 раз